In [29]:
# DeepFabric kütüphanesini GitHub'dan kuruyoruz
!pip install git+https://github.com/lukehinds/deepfabric.git -q

# Gerekli yardımcı kütüphaneleri kuruyoruz
!pip install datasets pandas matplotlib seaborn pyyaml -q

print("✅ Tüm kütüphaneler başarıyla kuruldu!")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Tüm kütüphaneler başarıyla kuruldu!


In [30]:
import json
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datasets import load_dataset
import subprocess
import os
from getpass import getpass

# Görselleştirme ayarları
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ Kütüphaneler başarıyla yüklendi!")

✅ Kütüphaneler başarıyla yüklendi!


In [31]:
# OpenRouter API anahtarını ayarla
OPENROUTER_API_KEY = getpass("OpenRouter API Anahtarınızı girin: ")

# Environment variable olarak ayarla
os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY

print("✅ API anahtarı ayarlandı!")

OpenRouter API Anahtarınızı girin: ··········
✅ API anahtarı ayarlandı!


In [33]:
# Yapılandırma dizini oluştur
config_dir = Path('deepfabric_configs')
config_dir.mkdir(exist_ok=True)

# Çıktı dizini oluştur
output_dir = Path('synthetic_datasets')
output_dir.mkdir(exist_ok=True)

print(f"✅ Dizinler oluşturuldu!")
print(f"   📁 Yapılandırma dizini: {config_dir}")
print(f"   📁 Çıktı dizini: {output_dir}")

✅ Dizinler oluşturuldu!
   📁 Yapılandırma dizini: deepfabric_configs
   📁 Çıktı dizini: synthetic_datasets


In [49]:
# Chain of Thought yapılandırması - OpenRouter + GPT-OSS-20B
cot_config = {
    'dataset_system_prompt': 'Sen bir matematik ve programlama uzmanısın. Öğrencilere adım adım problem çözmeyi öğretiyorsun. Her problem için düşünce sürecini detaylı açıklıyorsun.',

    'topic_graph': {
        'topic_prompt': 'Matematik ve Programlama Problemleri: Algoritmalar, veri yapıları, ve matematiksel çözümlemeler',
        'provider': 'openrouter',
        'model': 'openai/gpt-oss-20b',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.7,
        'degree': 3,  # Her düğümden 3 alt konu üretilecek
        'depth': 2,   # 2 seviye derinlik
        'save_as': str(output_dir / 'topics_cot.jsonl')
    },

    'data_engine': {
        'generation_system_prompt': 'Her problemi adım adım çöz ve düşünce sürecini açıkça göster.',
        'instructions': 'Adım adım akıl yürütme ile matematik ve programlama problemleri çöz.',
        'conversation_type': 'chain_of_thought',
        'reasoning_style' : 'structured',
        'provider': 'openrouter',
        'model': 'openai/gpt-oss-20b',
        'api_key': OPENROUTER_API_KEY,
        'base_url': 'https://openrouter.ai/api/v1',
        'temperature': 0.5,
        'max_retries': 3
    },

    'dataset': {
        'save_as': str(output_dir / 'cot_math_programming.jsonl'),
        'creation': {
            'num_steps': 1,  # 15 örnek üret
            'batch_size': 3,
            'sys_msg': True
        }
    }
}

# YAML dosyası olarak kaydet
cot_config_path = config_dir / 'cot_openrouter_config.yaml'
with open(cot_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(cot_config, f, allow_unicode=True)

print(f"✅ Chain of Thought yapılandırması kaydedildi: {cot_config_path}")
print("\n📋 Yapılandırma özeti:")
print(f"  • Konu derinliği: {cot_config['topic_graph']['depth']}")
print(f"  • Konu genişliği: {cot_config['topic_graph']['degree']}")
print(f"  • Örnek sayısı: {cot_config['dataset']['creation']['num_steps']}")
print(f"  • Batch boyutu: {cot_config['dataset']['creation']['batch_size']}")

✅ Chain of Thought yapılandırması kaydedildi: deepfabric_configs/cot_openrouter_config.yaml

📋 Yapılandırma özeti:
  • Konu derinliği: 2
  • Konu genişliği: 3
  • Örnek sayısı: 3
  • Batch boyutu: 3


In [50]:
# Chain of Thought verisi üret
cmd = [
    'deepfabric', 'generate',
    str(cot_config_path)
]

print("🚀 Chain of Thought verisi üretiliyor...")
print(f"📝 Komut: {' '.join(cmd)}")
print(f"📁 Config dosyası: {cot_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

# Çıktı dosyalarını kontrol et
output_file_cot = output_dir / 'cot_math_programming.jsonl'
if output_file_cot.exists():
    print("\n" + "="*80)
    print(f"✅ Chain of Thought verisi oluşturuldu: {output_file_cot}")
    print(f"📦 Dosya boyutu: {output_file_cot.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_cot}")

🚀 Chain of Thought verisi üretiliyor...
📝 Komut: deepfabric generate deepfabric_configs/cot_openrouter_config.yaml
📁 Config dosyası: deepfabric_configs/cot_openrouter_config.yaml


 📊 Path Validation Passed
   • Expected tree paths: ~9 (depth=2, degree=3)
   • Requested samples: 9 (3 steps × 3 batch size)
   • Path utilization: ~100.0%

╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│  DeepFabric Graph Generation                                                 │
│  Building interconnected topic structure with openai/gpt-oss-20b             │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Configuration: depth=2, degree=3

    Building graph - depth 2 (3 nodes to expand)     ━━━━━━━━━━━━━━━ 2/2 0:00:16
  Processing: ...ections":[0,1]},{"topic":"İstatistiksel Ana

In [51]:
# JSONL dosyasını oku
try:
    cot_df = pd.read_json(output_file_cot, lines=True)
    print("✅ JSONL dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(cot_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(cot_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {output_file_cot}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

✅ JSONL dosyası başarıyla okundu!

📋 DataFrame'in ilk 5 satırı:


,messages,reasoning,tool_context,agent_context,question,final_answer
0,"[{'role': 'system', 'content': 'Sen bir matema...","{'style': 'freetext', 'content': '...'}","{'available_tools': [{'name': '...', 'descript...","{'mode': 'single_turn', 'planning_trace': 'We ...",We need...,We need to produce ...
1,"[{'role': 'system', 'content': 'Sen bir matema...","{'style': 'structured', 'content': [{'step_num...",NaN,NaN,Bir tamsayı dizisinde en uzun artan alt diziyi...,The length of the longest increasing subsequen...



📊 Toplam satır sayısı: 2


In [54]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in cot_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in cot_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir matematik ve programlama uzmanısın. Öğrencilere adım adım problem çözmeyi öğretiyorsun. Her problem için düşünce sürecini detaylı açıklıyorsun.'}, {'role': 'user', 'content': '...'}]
Satır 1: [{'role': 'system', 'content': 'Sen bir matematik ve programlama uzmanısın. Öğrencilere adım adım problem çözmeyi öğretiyorsun. Her problem için düşünce sürecini detaylı açıklıyorsun.'}, {'role': 'user', 'content': 'Bir tamsayı dizisinde en uzun artan alt diziyi bul ve uzunluğunu döndür. Adım adım düşün.'}, {'role': 'assistant', 'content': 'The length of the longest increasing subsequence is 4.'}]

--- Sütun: reasoning ---
Satır 0: {'style': 'freetext', 'content': '...'}
Satır 1: {'style': 'structured', 'content': [{'step_number': 1, 'thought': 'Identify the problem: given an array of integers, find the length of the longest subsequence where each element is greater than the previous one.', 'act

In [37]:
#deepfabric --help
#conversation_type='chain_of_thought'. Choose from: 'freetext', 'structured',
#'hybrid'

In [56]:
print(cot_df.question.iloc[1])

Bir tamsayı dizisinde en uzun artan alt diziyi bul ve uzunluğunu döndür. Adım adım düşün.


In [58]:
# topics_cot.jsonl dosyasını oku
topics_cot_file = Path('/content/synthetic_datasets/topics_cot.jsonl')
try:
    topics_cot_df = pd.read_json(topics_cot_file, lines=True)
    print("✅ topics_cot.jsonl dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(topics_cot_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(topics_cot_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {topics_cot_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

⚠️ Dosya okunurken bir hata oluştu: Expected object or value


In [59]:
# topics_cot.jsonl dosyasının içeriğini görüntüle
topics_cot_file = Path('/content/synthetic_datasets/topics_cot.jsonl')

try:
    with open(topics_cot_file, 'r', encoding='utf-8') as f:
        content = f.read()
        print(content)

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {topics_cot_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

{
  "nodes": {
    "0": {
      "id": 0,
      "topic": "Matematik ve Programlama Problemleri: Algoritmalar, veri yapıları, ve matematiksel çözümlemeler",
      "children": [
        1,
        2,
        3,
        4,
        5,
        6,
        10,
        11,
        12
      ],
      "parents": []
    },
    "1": {
      "id": 1,
      "topic": "Algoritma Tasarımı ve Analizi: Temel Kavramlar, Süreçler ve Performans Ölçümleri",
      "children": [
        4,
        5,
        6,
        7,
        9,
        10,
        11
      ],
      "parents": [
        0
      ]
    },
    "2": {
      "id": 2,
      "topic": "Veri Yapıları: Yönlendirilmiş ve Çalışma Hızını Değiştiren ...",
      "children": [
        7,
        8,
        9
      ],
      "parents": [
        0
      ]
    },
    "3": {
      "id": 3,
      "topic": "Matematiksel Analiz ve Yetenek ...",
      "children": [
        10,
        11,
        12
      ],
      "parents": [
        0
      ]
    },
    "4": {
  

In [100]:
# Q&A yapılandırması
qa_config = {
    'dataset_system_prompt': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun. Her zaman Türkçe soru ve cevap vermelisin.',

    'topic_graph': {
        'topic_prompt': 'Python ve Makine Öğrenmesi: Temel kavramlar, kütüphaneler, algoritmalar ve uygulamalar',
        'provider': 'openrouter',
        'model': 'deepseek/deepseek-v3.2-exp',
        'temperature': 0.7,
        'degree': 2,
        'depth': 1,
        'save_as': str(output_dir / 'topics_qa.jsonl')
    },

    'data_engine': {
        'generation_system_prompt': 'Net ve açıklayıcı cevaplar ver.',
        'instructions': 'Python ve makine öğrenmesi konularında eğitim amaçlı Q&A çiftleri oluştur.',
        'conversation_type': 'basic',
        'provider': 'openrouter',
        'model': 'deepseek/deepseek-v3.2-exp',
        'temperature': 0.4,
        'max_retries': 3
    },

    'dataset': {
        'save_as': str(output_dir / 'qa_python_ml.jsonl'),
        'creation': {
            'num_steps': 2,
            'batch_size': 1,
            'sys_msg': True
        }
    }
}

qa_config_path = config_dir / 'qa_config.yaml'
with open(qa_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(qa_config, f, allow_unicode=True)

print(f"✅ Q&A yapılandırması kaydedildi: {qa_config_path}")

✅ Q&A yapılandırması kaydedildi: deepfabric_configs/qa_config.yaml


In [104]:
# Q&A verisi üret
cmd = [
    'deepfabric', 'generate',
    str(qa_config_path)
]

print("🚀 Q&A verisi üretiliyor...")
print(f"📁 Config dosyası: {qa_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_qa = output_dir / 'qa_python_ml.jsonl'
if output_file_qa.exists():
    print("\n" + "="*80)
    print(f"✅ Q&A verisi oluşturuldu: {output_file_qa}")
    print(f"📦 Dosya boyutu: {output_file_qa.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_qa}")

🚀 Q&A verisi üretiliyor...
📁 Config dosyası: deepfabric_configs/qa_config.yaml


 📊 Path Validation Passed
   • Expected tree paths: ~2 (depth=1, degree=2)
   • Requested samples: 2 (2 steps × 1 batch size)
   • Path utilization: ~100.0%

╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│  DeepFabric Graph Generation                                                 │
│  Building interconnected topic structure with deepseek/deepseek-v3.2-exp     │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Configuration: depth=1, degree=2

    Building graph - depth 1 (1 nodes to expand)     ━━━━━━━━━━━━━━━ 1/1 0:00:05
  Processing: ... Denetimsiz Öğrenme, Pekiştirmeli Öğrenme", "connections": [0]…
╭────────────────────────────── Final Statistics ──────────────────────────────

In [105]:
# qa_python_ml.jsonl dosyasını oku
qa_python_ml_file = Path('/content/synthetic_datasets/qa_python_ml.jsonl')

try:
    qa_df = pd.read_json(qa_python_ml_file, lines=True)
    print("✅ qa_python_ml.jsonl dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(qa_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(qa_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {qa_python_ml_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

✅ qa_python_ml.jsonl dosyası başarıyla okundu!

📋 DataFrame'in ilk 5 satırı:


,messages,question,final_answer
0,"[{'role': 'system', 'content': 'Sen bir eğitim...",Denetimli öğrenme ve denetimsiz öğrenme arasın...,Denetimli öğrenme etiketli verilerle tahmin ya...



📊 Toplam satır sayısı: 1


In [102]:
# qa_python_ml.jsonl dosyasını oku
qa_python_ml_file = Path('/content/synthetic_datasets/qa_python_ml.jsonl')

try:
    qa_df = pd.read_json(qa_python_ml_file, lines=True)
    print("✅ qa_python_ml.jsonl dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(qa_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(qa_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {qa_python_ml_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

✅ qa_python_ml.jsonl dosyası başarıyla okundu!

📋 DataFrame'in ilk 5 satırı:


,messages,question,final_answer
0,"[{'role': 'system', 'content': 'Sen bir eğitim...",Pandas kütüphanesinde DataFrame'ler arasında n...,Pandas kütüphanesinde DataFrame'leri birleştir...



📊 Toplam satır sayısı: 1


In [96]:
# qa_python_ml.jsonl dosyasını oku
qa_python_ml_file = Path('/content/synthetic_datasets/qa_python_ml.jsonl')

try:
    qa_df = pd.read_json(qa_python_ml_file, lines=True)
    print("✅ qa_python_ml.jsonl dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(qa_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(qa_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {qa_python_ml_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

✅ qa_python_ml.jsonl dosyası başarıyla okundu!

📋 DataFrame'in ilk 5 satırı:


,messages,question,final_answer
0,"[{'role': 'system', 'content': 'Sen bir eğitim...",Lineer Regresyon'un temel amacını ve matematik...,"Lineer Regresyon, sürekli bir çıktı değişkenin..."



📊 Toplam satır sayısı: 1


In [97]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in qa_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in qa_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun. Her zaman Türkçe soru ve cevap vermelisin.'}, {'role': 'user', 'content': "Lineer Regresyon'un temel amacını ve matematiksel formülünü basit bir şekilde açıklar mısın? Ayrıca, Python'da scikit-learn kütüphanesi ile basit bir lineer regresyon modeli nasıl uygulanır, kısaca gösterir misin?"}, {'role': 'assistant', 'content': 'Lineer Regresyon, sürekli bir çıktı değişkenini (hedef değişken) bir veya daha fazla girdi değişkeninden (özellikler) tahmin etmek için kullanılan denetimli bir öğrenme algoritmasıdır. Temel amacı, veri noktalarına en iyi şekilde uyan doğrusal bir ilişki (düz bir çizgi) bulmaktır.\n\nMatematiksel olarak, basit lineer regresyon (tek bir özellikle) şu formülle ifade edilir:\n`y = b0 + b1*x`\nBurada:\n- `y`: Tahmin edilen hedef değerdir.\n- `x`: Girdi özelliğidir.\n- `b0`: Y-kesim 

In [89]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in qa_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in qa_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun.'}, {'role': 'user', 'content': 'Scikit-learn kütüphanesi ne için kullanılır ve temel özellikleri nelerdir?'}, {'role': 'assistant', 'content': "Scikit-learn, Python'da makine öğrenmesi için yaygın olarak kullanılan, açık kaynaklı ve kapsamlı bir kütüphanedir. Temel kullanım alanları ve özelliklerini şu şekilde özetleyebiliriz:\n\n**Temel Kullanım Alanları:**\n1. **Sınıflandırma (Classification):** Veri noktalarını önceden tanımlanmış kategorilere ayırmak için (örn. e-posta spam mi değil mi?).\n2. **Regresyon (Regression):** Sürekli bir çıktı değişkenini tahmin etmek için (örn. ev fiyatı tahmini).\n3. **Kümeleme (Clustering):** Etiketlenmemiş verileri benzerliklerine göre gruplara ayırmak için.\n4. **Boyut İndirgeme (Dimensionality Reduction):** Veri kümesindeki değişken sayısını azaltarak işlem kar

In [103]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in qa_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in qa_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun. Her zaman Türkçe soru ve cevap vermelisin.'}, {'role': 'user', 'content': "Pandas kütüphanesinde DataFrame'ler arasında nasıl birleştirme işlemi yapılır? Farklı birleştirme türlerini açıklar mısın?"}, {'role': 'assistant', 'content': "Pandas kütüphanesinde DataFrame'leri birleştirmek için başlıca dört yöntem bulunur: merge(), join(), concat() ve append().\n\n1. merge(): SQL'deki JOIN işlemine benzer şekilde çalışır. İki DataFrame'i belirli bir sütuna göre birleştirir. Ana birleştirme türleri:\n - inner: Sadece ortak anahtarlara sahip satırları birleştirir\n - outer: Tüm satırları birleştirir, eksik değerler NaN olur\n - left: Sol DataFrame'deki tüm satırları korur\n - right: Sağ DataFrame'deki tüm satırları korur\n\n2. join(): DataFrame'leri indekslerine göre birleştirir, merge()'ün özel bir halidi

In [106]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in qa_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in qa_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun. Her zaman Türkçe soru ve cevap vermelisin.'}, {'role': 'user', 'content': 'Denetimli öğrenme ve denetimsiz öğrenme arasındaki temel farklar nelerdir? Her birine birer örnek vererek açıklar mısın?'}, {'role': 'assistant', 'content': "Denetimli öğrenme ve denetimsiz öğrenme, makine öğrenmesinin iki temel kategorisidir ve temel farkları şunlardır:\n\n**Denetimli Öğrenme (Supervised Learning):**\n- Etiketli verilerle çalışır (girdi ve çıktı çiftleri)\n- Model, girdilerden çıktıları tahmin etmeyi öğrenir\n- Temel amacı haritalama fonksiyonunu öğrenmektir\n- Örnek: E-posta spam filtresi - model, e-postaları 'spam' veya 'spam değil' olarak sınıflandırmayı öğrenir\n\n**Denetimsiz Öğrenme (Unsupervised Learning):**\n- Etiketsiz verilerle çalışır (sadece girdi verileri)\n- Model, verideki gizli yapıları veya

In [107]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in qa_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in qa_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen bir eğitim asistanısın. Kullanıcı sorularına net, açıklayıcı ve doğru cevaplar veriyorsun. Her zaman Türkçe soru ve cevap vermelisin.'}, {'role': 'user', 'content': 'Denetimli öğrenme ve denetimsiz öğrenme arasındaki temel farklar nelerdir? Her birine birer örnek vererek açıklar mısın?'}, {'role': 'assistant', 'content': "Denetimli öğrenme ve denetimsiz öğrenme, makine öğrenmesinin iki temel kategorisidir ve temel farkları şunlardır:\n\n**Denetimli Öğrenme (Supervised Learning):**\n- Etiketli verilerle çalışır (girdi ve çıktı çiftleri)\n- Model, girdilerden çıktıları tahmin etmeyi öğrenir\n- Temel amacı haritalama fonksiyonunu öğrenmektir\n- Örnek: E-posta spam filtresi - model, e-postaları 'spam' veya 'spam değil' olarak sınıflandırmayı öğrenir\n\n**Denetimsiz Öğrenme (Unsupervised Learning):**\n- Etiketsiz verilerle çalışır (sadece girdi verileri)\n- Model, verideki gizli yapıları veya

In [108]:
# Multi-turn conversation yapılandırması
multiturn_config = {
    'dataset_system_prompt': 'Sen yardımcı bir teknoloji danışmanısın. Kullanıcılarla doğal, çok turlu konuşmalar yaparak sorunlarını adım adım çözüyorsun.',

    'topic_graph': {
        'topic_prompt': 'Yazılım Geliştirme Destek: Debugging, kod optimizasyonu, mimari kararlar ve best practices',
        'provider': 'openrouter',
        'model': 'deepseek/deepseek-v3.2-exp',
        'temperature': 0.7,
        'degree': 3,
        'depth': 1,
        'save_as': str(output_dir / 'topics_multiturn.jsonl')
    },

    'data_engine': {
        'generation_system_prompt': 'Her turda bağlamı hatırlayarak yardımcı ol ve problemi adım adım çöz.',
        'instructions': 'Kullanıcıyla çok turlu, doğal konuşmalar yap.',
        #'conversation_type': 'basic',
        'provider': 'openrouter',
        'model': 'deepseek/deepseek-v3.2-exp',
        'temperature': 0.6,
        'max_retries': 3
    },

    'dataset': {
        'save_as': str(output_dir / 'multiturn_tech_support.jsonl'),
        'creation': {
            'num_steps': 3,
            'batch_size': 1,
            'sys_msg': True
        }
    }
}

multiturn_config_path = config_dir / 'multiturn_config.yaml'
with open(multiturn_config_path, 'w', encoding='utf-8') as f:
    yaml.dump(multiturn_config, f, allow_unicode=True)

print(f"✅ Multi-turn yapılandırması kaydedildi: {multiturn_config_path}")

✅ Multi-turn yapılandırması kaydedildi: deepfabric_configs/multiturn_config.yaml


In [109]:
# Multi-turn verisi üret
cmd = [
    'deepfabric', 'generate',
    str(multiturn_config_path)
]

print("🚀 Multi-turn conversation verisi üretiliyor...")
print(f"📁 Config dosyası: {multiturn_config_path}")
print("\n" + "="*80 + "\n")

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("⚠️ Uyarılar:")
    print(result.stderr)

output_file_multiturn = output_dir / 'multiturn_tech_support.jsonl'
if output_file_multiturn.exists():
    print("\n" + "="*80)
    print(f"✅ Multi-turn verisi oluşturuldu: {output_file_multiturn}")
    print(f"📦 Dosya boyutu: {output_file_multiturn.stat().st_size / 1024:.2f} KB")
else:
    print("\n" + "="*80)
    print(f"⚠️ Çıktı dosyası bulunamadı: {output_file_multiturn}")

🚀 Multi-turn conversation verisi üretiliyor...
📁 Config dosyası: deepfabric_configs/multiturn_config.yaml


 📊 Path Validation Passed
   • Expected tree paths: ~3 (depth=1, degree=3)
   • Requested samples: 3 (3 steps × 1 batch size)
   • Path utilization: ~100.0%

╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│  DeepFabric Graph Generation                                                 │
│  Building interconnected topic structure with deepseek/deepseek-v3.2-exp     │
│                                                                              │
╰──────────────────────────────────────────────────────────────────────────────╯
Configuration: depth=1, degree=3

    Building graph - depth 1 (1 nodes to expand)     ━━━━━━━━━━━━━━━ 1/1 0:00:04
  Processing: ... "topic": "Yazılım Mimarisi Desenleri", "connections": [0] } ]…
╭────────────────────────────── Final Statistics ───

In [110]:
# multiturn_tech_support.jsonl dosyasını oku
multiturn_tech_support_file = Path('/content/synthetic_datasets/multiturn_tech_support.jsonl')

try:
    multiturn_df = pd.read_json(multiturn_tech_support_file, lines=True)
    print("✅ multiturn_tech_support.jsonl dosyası başarıyla okundu!")
    print("\n📋 DataFrame'in ilk 5 satırı:")
    display(multiturn_df.head())
    print(f"\n📊 Toplam satır sayısı: {len(multiturn_df)}")

except FileNotFoundError:
    print(f"⚠️ Hata: Dosya bulunamadı - {multiturn_tech_support_file}")
except Exception as e:
    print(f"⚠️ Dosya okunurken bir hata oluştu: {e}")

✅ multiturn_tech_support.jsonl dosyası başarıyla okundu!

📋 DataFrame'in ilk 5 satırı:


,messages,reasoning,question,final_answer
0,"[{'role': 'system', 'content': 'Sen yardımcı b...","{'style': 'structured', 'content': [{'step_num...",Python'da büyük bir veri listesi üzerinde filt...,Python'da büyük veri kümeleri için optimizasyo...
1,"[{'role': 'system', 'content': 'Sen yardımcı b...","{'style': 'structured', 'content': [{'step_num...",Singleton tasarım desenini ne zaman kullanmam ...,"Singleton tasarım deseni, bir sınıfın yalnızca..."



📊 Toplam satır sayısı: 2


In [111]:
# Her sütundaki veriyi yazdırma
print("📊 DataFrame Sütun İçerikleri:")
for col in multiturn_df.columns:
    print(f"\n--- Sütun: {col} ---")
    try:
        # Tüm girdileri yazdır
        for index, value in multiturn_df[col].items():
             print(f"Satır {index}: {value}")

    except Exception as e:
        print(f"Bu sütundaki veriyi yazdırırken bir hata oluştu: {e}")

📊 DataFrame Sütun İçerikleri:

--- Sütun: messages ---
Satır 0: [{'role': 'system', 'content': 'Sen yardımcı bir teknoloji danışmanısın. Kullanıcılarla doğal, çok turlu konuşmalar yaparak sorunlarını adım adım çözüyorsun.'}, {'role': 'user', 'content': "Python'da büyük bir veri listesi üzerinde filtreleme yapmam gerekiyor. Şu an for döngüsü kullanıyorum ama performans çok yavaş. Daha hızlı bir yöntem önerebilir misin?"}, {'role': 'assistant', 'content': "Tabii ki! Öncelikle mevcut kodunu görebilir miyim? Böylece senin özel durumuna uygun optimizasyon önerebilirim. Genel olarak, Python'da büyük veri kümeleri için list comprehension yerine generator expression kullanmak, NumPy array'lerine geçmek veya pandas DataFrame kullanmak gibi optimizasyonlar düşünebiliriz."}]
Satır 1: [{'role': 'system', 'content': 'Sen yardımcı bir teknoloji danışmanısın. Kullanıcılarla doğal, çok turlu konuşmalar yaparak sorunlarını adım adım çözüyorsun.'}, {'role': 'user', 'content': 'Merhaba, bir e-ticaret uyg